In [1]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "Qwen/Qwen2.5-7B-Instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# prompt = "Give me a short introduction to large language model."
# messages = [
#     {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
# model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# generated_ids = model.generate(
#     **model_inputs,
#     max_new_tokens=512
# )
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

# response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [1]:
import pandas as pd 

df_train = pd.read_csv("../xfdata/train.csv", sep="\t")
df_test = pd.read_csv("../xfdata/test_submit.csv", sep="\t")

In [10]:
import json 
import re

def split_into_sentences(sentence):
    # 使用正则表达式根据句号、问号、感叹号进行分割
    sentence_list = re.split(r'(?<=[。！？])', sentence)
    # 去掉空字符串和前后空格
    sentence_list = [s.strip() for s in sentence_list if s.strip()]
    return sentence_list
    

multi_task = []
for content, person, sentiment, en in df_train.values:
    # 实体抽取（人名提取）
    if str(person) != "nan":
        person = person.split(",")
        sentences = split_into_sentences(content)
        for sentence in sentences:
            name_list = [i for i in person if i in sentence]
            # 获取人名的首次出现位置
            name_positions = [(name, sentence.find(name)) for name in name_list if sentence.find(name) != -1]
            # 根据出现位置排序
            sorted_names = sorted(name_positions, key=lambda x: x[1])
            # 提取排序后的名字列表
            sorted_name_list = [name for name, _ in sorted_names]
            output = ",".join(sorted_name_list)
            multi_task.append({
                "instruction": "请基于以下内容，完成实体抽取（人名提取）。",
                "input": sentence,
                "output": output if output else "无"
            })
    # 情感分析（正向或负向）
    multi_task.append({
        "instruction": "请基于以下内容，完成情感分析（正向或负向）。",
        "input": content,
        "output": sentiment
    })
    # 文本翻译（中文翻译为英文）
    multi_task.append({
        "instruction": "请基于以下内容，完成文本翻译（中文翻译为英文）",
        "input": content,
        "output": en
    })

with open("../user_data/LLaMA-Factory/data/multi_task.json", "w", encoding="utf-8") as f:
    json.dump(multi_task, f, indent=4, ensure_ascii=False)

# LoRA微调后进行预测

In [2]:
from lmdeploy import pipeline, GenerationConfig, TurbomindEngineConfig
from transformers import AutoTokenizer
import nest_asyncio

nest_asyncio.apply()

# backend_config = TurbomindEngineConfig(
#         rope_scaling_factor=2.5,
#         session_len=1048576,  # 1M context length
#         max_batch_size=1,
#         cache_max_entry_count=0.7,
# ) 
pipe = pipeline("/root/onethingai-tmp/user_data/LLaMA-Factory/models/qwen2_5_lora_sft")
tokenizer = AutoTokenizer.from_pretrained("/root/onethingai-tmp/user_data/LLaMA-Factory/models/qwen2_5_lora_sft", trust_remote_code=True)

[WARNING] gemm_config.in is not found; using default GEMM algo


In [3]:
import pandas as pd 
import json 
import re


df_train = pd.read_csv("../xfdata/train.csv", sep="\t")
df_test = pd.read_csv("../xfdata/test_submit.csv", sep="\t")

def split_into_sentences(sentence):
    # 使用正则表达式根据句号、问号、感叹号进行分割
    sentence_list = re.split(r'(?<=[。！？])', sentence)
    # 去掉空字符串和前后空格
    sentence_list = [s.strip() for s in sentence_list if s.strip()]
    return sentence_list

In [4]:
from tqdm import tqdm 


result = []
for content in tqdm(df_test["content"]):
    try:
        # print(f"==================================================[{idx}|{len(df_test)}]==================================================")
        result_ = []
        # 实体抽取（人名提取）
        sentences = split_into_sentences(content)
        person = []
        for sentence in sentences:
            conversation = [
                {
                    "role": "system",
                    "content": "请基于以下内容，完成实体抽取（人名提取）。"
                },
                {
                    "role": "user",
                    "content": sentence
                }
            ]
            response = pipe(tokenizer.decode(tokenizer.apply_chat_template(conversation)) + " \n").text
            # print(f"sentence: {sentence}")
            # print(f"person: {response}")
            if response != "无":
                person.extend(response.split(","))
        name_list = [i for i in set(person) if i in content]
        # 获取人名的首次出现位置
        name_positions = [(name, content.find(name)) for name in name_list if content.find(name) != -1]
        # 根据出现位置排序
        sorted_names = sorted(name_positions, key=lambda x: x[1])
        # 提取排序后的名字列表
        sorted_name_list = [name for name, _ in sorted_names]
        person = ",".join(sorted_name_list)
        if person == "无":
            person = np.NaN
        result_.append(person)
    
        # 情感分析（正向或负向）
        # response = model.chat(tokenizer, content, [], meta_instruction="请基于以下内容，完成情感分析（正向或负向）。")
        conversation = [
                {
                    "role": "system",
                    "content": "请基于以下内容，完成情感分析（正向或负向）。"
                },
                {
                    "role": "user",
                    "content": content
                }
            ]
        response = pipe(tokenizer.decode(tokenizer.apply_chat_template(conversation)) + " \n").text
        # print(f"content: {content}")
        # print(f"sentiment: {response}")
        result_.append(response)
    
        # 文本翻译（中文翻译为英文）
        # response = model.chat(tokenizer, content, [], meta_instruction="请基于以下内容，完成文本翻译（中文翻译为英文）")
        trans = ""
        for sentence in sentences:
            conversation = [
                {
                    "role": "system",
                    "content": "请基于以下内容，完成文本翻译（中文翻译为英文）"
                },
                {
                    "role": "user",
                    "content": sentence
                }
            ]
            response = pipe(tokenizer.decode(tokenizer.apply_chat_template(conversation)) + " \n").text
            if response[-1] == " ":
                trans += response
            else:
                trans += response + " "
        trans = trans.strip()
        result_.append(trans)
        print(result_)
        result.append(result_)
    except:
        result.append(["", "", ""])

  0%|          | 0/536 [00:00<?, ?it/s]2024-09-23 13:03:07.379902: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-23 13:03:07.406349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 13:03:07.430246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 13:03:07.437054: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 13:03:07.457339: I te

['', '正向', "The upward trend is on schedule, with 3180 as the first goal for the end of the year. Huaxun Financial will welcome a red start in November. Presenting a pattern of increased volume and overall gains in individual stocks. Affected by the sharp decline of the US dollar, the two markets opened high and quickly advanced to the 3000-point level, successfully recovering the 10-day and 5-day moving averages. From the chart, the Growth Enterprise Market was not affected by the first batch of 1.2 billion shares being unlocked, but instead, it rose sharply, which undoubtedly boosted market sentiment. Permanent magnet, gold, multicrystalline silicon, low carbon and other concepts surged, and the strong pull of non-ferrous metals and coal sectors has also attracted a large amount of funds from outside the market. Meanwhile, the Tibet, Inner Mongolia, Qinghai and other regional plates also showed much stronger performance than the market. Affected by the further tightening of regulated

  0%|          | 2/536 [00:26<1:52:06, 12.60s/it]

['哈马斯,巴尔胡姆,卡桑,佩雷斯,阿巴斯,阿布-鲁代纳,萧熙', '正向', 'Hamas vows to launch a third intifada to crush the Israeli regime. China media, December 28, 2022. According to the Israeli media on the 28th, the Israeli military launched a large-scale air strike on the Gaza Strip on the 27th, resulting in hundreds of casualties, and Hamas\' office buildings and multiple camps were also severely damaged. Hamas armed forces issued a statement on the night of the 27th, vowing to launch a "third uprising" to thoroughly defeat the Israeli regime. In a statement released by Hamas armed forces on the evening of the 27th, it said that Israel launched a military operation similar to a terrorist attack against the civilian targets of Palestine, which has already caused hundreds of innocent civilians to be injured and killed, and Hamas will retaliate against Israel with "hellish" revenge and continue to launch rockets into Israeli territory. The statement also threatened that war had begun, and the time for Hamas to lau

  1%|          | 3/536 [00:32<1:27:34,  9.86s/it]

...

['赵静', '正向', 'The first-level discipline of doctoral authorization in Qinghai Province has achieved a breakthrough of zero. On May 9, the reporter from the Department of Higher Education of the Education Department of Qinghai Province learned that the State Council Degree Committee recently issued the "Notice on the List of First-level Disciplines of Doctoral and Master\'s Degrees Authorized for Review and Addition in 2010", officially announcing the list of first-level disciplines authorized by each degree-granting unit that has been approved in the new round of degree authorization review work launched in 2010. Our province has made a significant breakthrough in the addition of doctoral and master\'s degree authorization for first-level disciplines. Qinghai University\'s crop science discipline has obtained the authorization for a doctoral point, achieving a breakthrough of zero for first-level disciplines with doctoral authorization in our province. 24 new master\'s degree authoriza

100%|█████████▉| 535/536 [1:28:38<00:06,  6.44s/it]

['康元', '正向', "I83205230 Competition 1500 yuan within the smartphone recommendation / Kang Yuan Smartphones have not been late, but have not caused a great response, because its high price has prevented many people from buying, but now there are relatively low models of various systems, in order to further popularize smart phones, and some models are even below 1000 yuan, which has a strong appeal to students and some friends with low requirements. Below, the author will recommend 8 models within 1500 yuan, all of which are quite popular in the market. 500W capacitive touch: Samsung i8320 (parameters forum) reference quotation: 1480 yuan (water machine) Recently, the most popular mobile phone in the modified machine market is this i8320 model. Although the price is very cheap, it has high-end configurations such as 500W pixel 720P HD, 3.5 inch WVGA capacitive touch and 600MHz processor. It uses the latest Li-Mo system, and the configuration is very good. Editor's review: As the first mo

100%|██████████| 536/536 [1:28:44<00:00,  9.93s/it]

['李国辉,李文源,郭某,邵某', '负向', 'A man forcibly occupied others\' seats on the train and charged a seat change fee, and the new快报 (Reporter Li Guohui, Li Wenyuan, a reporter from the Liwenyuan) Two Hebei men, Guo Mou and Shao Mou, who failed to find a job in Guangzhou, attempted to make money by forcibly occupying others\' seats on the train and charging a seat change fee, and were caught by the train police, and were punished with administrative detention and fines respectively. At 6:00 a.m. on the morning of the 16th, a passenger rushed to the duty room of the train police after the L308 train left Guangzhou station, and reported that someone had occupied his seat and was forcibly charging a seat-letting fee. The train police immediately rushed to the carriage where the incident occurred and apprehended two suspects, confiscating 130 yuan in illegal proceeds on the spot. It was found that Guo and Shao, the suspects, were both in their early thirties. At the beginning of this month, they came

In [5]:
df_test[["person", "sentiment", "en"]] = result
df_test.to_csv("../prediction_result/submit.csv", sep="\t", index=False)